# Import libraries

In [8]:
# uncomment to install libraries
# ! pip install numpy pandas requests bs4 regex glob2 geopy
# ! pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install 

import numpy as np
import pandas as pd
import glob
import requests
from bs4 import BeautifulSoup
import re
import time
import geopy

In [9]:
dissimilarity_matrix = pd.read_csv("data/dissimilarity_matrix.csv", index_col = ["city_state_company_room"])

,"Auburn, Alabama - Auburn Escape Zones: The Cabin","Auburn, Alabama - Auburn Escape Zones: Imprisoned","Auburn, Alabama - Auburn Escape Zones: Vault","Auburn, Alabama - Auburn Escape Zones: The Puzzler vs Superheroes","Auburn, Alabama - Auburn Escape Zones: Black Beard's Brig","Birmingham, Alabama - Breakout Games: The Kidnapping","Birmingham, Alabama - Breakout Games: Runaway Train","Birmingham, Alabama - Breakout Games: Museum Heist","Birmingham, Alabama - Breakout Games: Do Not Disturb","Birmingham, Alabama - Breakout Games: Mystery Mansion",...,"Evanston, Wyoming - Brain Drain: Jungle Mystery","Evanston, Wyoming - Brain Drain: Portal","Evanston, Wyoming - Escape Room Enigma: Tesla","Gillette, Wyoming - 307 Mysteries: Justice Is Blind","Gillette, Wyoming - 307 Mysteries: Commie Trickery","Gillette, Wyoming - 307 Mysteries: Wacked Science","Gillette, Wyoming - 307 Mysteries: Lifting The Jewels","Gillette, Wyoming - 307 Mysteries: Miss Molly's Zombie Reboot","Sheridan, Wyoming - Escapade: Mystic Mistake","Sheridan, Wyoming - Escapade: Sherlock and the Dragon’s Eye"
city_state_company_room,,,,,,,,,,,,,,,,,,,,,
"Auburn, Alabama - Auburn Escape Zones: The Cabin",0.000000,1.428897,2.698540,0.582664,1.298856,2.121373,2.121373,2.227071,3.440306,2.227071,...,3.180231,3.245182,3.128805,2.273314,2.273314,2.121373,2.398380,2.273314,2.557677,2.423629
"Auburn, Alabama - Auburn Escape Zones: Imprisoned",1.428897,0.000000,2.520205,1.513657,0.675329,1.889323,1.889323,2.972252,4.529379,2.972252,...,4.335140,4.383009,2.456828,2.058464,2.058464,1.889323,2.195801,2.058464,2.368760,2.223352
"Auburn, Alabama - Auburn Escape Zones: Vault",2.698540,2.520205,0.000000,2.769551,2.357447,2.864981,2.864981,3.353998,4.550465,3.353998,...,4.357167,4.404797,3.087266,2.979236,2.979236,2.864981,3.075730,2.979236,2.180504,2.021599
"Auburn, Alabama - Auburn Escape Zones: The Puzzler vs Superheroes",0.582664,1.513657,2.769551,0.000000,1.427510,2.285254,2.285254,2.351261,3.594224,2.351261,...,3.192963,3.504523,3.134465,2.560830,2.560830,2.285254,2.734224,2.560830,2.816310,2.568294
"Auburn, Alabama - Auburn Escape Zones: Black Beard's Brig",1.298856,0.675329,2.357447,1.427510,0.000000,1.666032,1.666032,2.533335,4.059082,2.533335,...,3.841134,3.895080,2.901327,1.855639,1.855639,1.666032,2.006904,1.855639,2.194800,2.037011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Gillette, Wyoming - 307 Mysteries: Wacked Science",2.121373,1.889323,2.864981,2.285254,1.666032,0.000000,0.000000,1.791123,3.535338,1.791123,...,3.729240,3.672859,2.642921,0.577812,0.577812,0.000000,0.866718,0.577812,1.306758,1.172071
"Gillette, Wyoming - 307 Mysteries: Lifting The Jewels",2.398380,2.195801,3.075730,2.734224,2.006904,0.866718,0.866718,2.111900,3.640030,2.111900,...,4.020053,3.638612,2.870023,0.288906,0.288906,0.866718,0.000000,0.288906,1.207153,1.457721
"Gillette, Wyoming - 307 Mysteries: Miss Molly's Zombie Reboot",2.273314,2.058464,2.979236,2.560830,1.855639,0.577812,0.577812,1.968719,3.582246,1.968719,...,3.904189,3.627124,2.766363,0.000000,0.000000,0.577812,0.288906,0.000000,1.172071,1.306758


In [12]:
(dissimilarity_matrix["Auburn, Alabama - Auburn Escape Zones: The Cabin"]
                    .sort_values()
                    .to_frame()
                    .reset_index(level = "city_state_company_room")
                    .rename(columns = {"Auburn, Alabama - Auburn Escape Zones: The Cabin": "dissimilarity"}))

,city_state_company_room,dissimilarity
0,"Auburn, Alabama - Auburn Escape Zones: The Cabin",0.000000
1,"Auburn, Alabama - Auburn Escape Zones: The Puz...",0.582664
2,"Charlotte, North Carolina - Escape Tactic: Min...",0.780872
3,"Carrollton, Georgia - Lock City Escape Games: ...",0.780872
4,"Watertown, New York - The Escape Works: Family...",0.976877
...,...,...
7912,"Nashville, Tennessee - The Escape Game: Team B...",12.886451
7913,"Bettendorf, Iowa - Skellington Manor: Prohibition",17.163203
7914,"Orlando, Florida - The Escape Game: Team Build...",19.470451
7915,"Kauai, Hawaii - Kauai Escape Room: Captain Mad...",39.018183


# Define functions

In [3]:
# recode difficulty_level
def recode_difficulty(x):
    if x == "Very easy":
        return 0
    elif x == "Easy":
        return 1
    elif x == "Average":
        return 2
    elif x == "Difficult":
        return 3
    elif x == "Very difficult":
        return 4
    else:
        return "None"

# recode fear_level
def recode_fear(x):
    if x == "Not scary":
        return 0
    elif x == "A little scary":
        return 1
    elif x == "Scary":
        return 2
    elif x == "Very scary":
        return 3
    else:
        return "None"

# minumum age
def code_minimum_age(x):
    if "+" in str(x):
        return str(x).strip("+")
    else:
        return str(x).split(" - ")[0]

# Geocoder using the Google Maps v3 API
api_key = open("google_maps_api_key.txt").readlines()[0]
googlev3_locator = geopy.geocoders.GoogleV3(api_key = api_key, timeout = 1000)

# address, latitude, and longitude
def get_lat_long(address):
    # look up location
    location = googlev3_locator.geocode(address)
    return location.address, location.latitude, location.longitude

# Test web scraping escape room reviews

## World of Escapes map page URL

In [3]:
map_page_url = "https://worldofescapes.com/map"

# also home page
home_page_url = "https://worldofescapes.com"

## Map page response

In [4]:
map_page_response = requests.get(map_page_url)

## Map page soup

In [5]:
map_page_soup = BeautifulSoup(map_page_response.text, "html.parser")

## Room URLs within cities within states

In [6]:
# dictionary containers for room urls within cities within states
state_city_room_urls = {}

# loop through rooms within cities within states
for state_item_i in map_page_soup.find_all("div", {"class": "col-lg-3 col-md-4 col-sm-6 col-xs-6 state-item"}):
    # state string
    state_i = state_item_i.find("h3").get_text().strip()
    
    # add state-level dictionary
    state_city_room_urls[state_i] = {}
    
    # add cities to state values
    for city_item_j in state_item_i.find_all("li", {"class": "city-item"}):
        # city string
        city_j = city_item_j.find("a").get_text()
        
        # add city-level dictionary to the state-level dictionary
        state_city_room_urls[state_i][city_j] = {}
        
        # city url
        city_url_j = home_page_url + city_item_j.find("a")["href"]
        
        # city url response
        city_response_j = requests.get(city_url_j)
        
        # city url soup
        city_soup_j = BeautifulSoup(city_response_j.text, "html.parser")
        
        # sleep for between 1-20 seconds
        sleep_time_j = np.random.randint(low = 1, high = 20, size = 1)
        
        # print stage in loop
        print("Made soup for the {}, {} page. Sleeping for {} seconds.".format(city_j, state_i, str(sleep_time_j).strip("[]")))
        
        # now sleep
        time.sleep(sleep_time_j)
        
        # list containers for escape room company name, escape room game name, and url
        company_name = []
        room_name = []
        room_url = []
                
        # company title
        for company_link in city_soup_j.find_all("a", {"class": "company-link"}):
            company_name.append(str(re.findall('title="(.*?)"', str(company_link))).strip("['']"))
                
        # game title
        for room_link in city_soup_j.find_all("a", {"class": "quest_tile_name_link"}):
            room_name.append(room_link.get_text())
                    
        # concatenate company name: room name
        company_and_room = [": ".join([company, room]) for company, room in zip(company_name, room_name)]
                
        # add lists for company-room labels
        for company_and_room_k in company_and_room:
            state_city_room_urls[state_i][city_j][company_and_room_k] = []
                
        # loop through href for each room, append to home page to make url
        if city_soup_j.find("section", {"class": "container"}) is not None:
            for a in city_soup_j.find("section", {"class": "container"}).find_all("a", {"class": "item-hover quest_tile_hover_link"}):
                room_url.append(home_page_url + a["href"])
                
            # append room urls to room dictionaries
            for company_and_room_k, room_url_k in zip(company_and_room, room_url):
                state_city_room_urls[state_i][city_j][company_and_room_k].append(room_url_k)
                
                # print loop result
                print("Saved URL for {} in {}, {}.".format(company_and_room_k, city_j, state_i))
        else:
            print(None)

Made soup for the Auburn, Alabama page. Sleeping for 8 seconds.
Saved URL for Auburn Escape Zones: The Cabin in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Imprisoned in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Vault in Auburn, Alabama.
Saved URL for Auburn Escape Zones: The Puzzler vs Superheroes in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Black Beard's Brig in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Vault in Auburn, Alabama.
Saved URL for Auburn Escape Zones: The Puzzler vs Superheroes in Auburn, Alabama.
Saved URL for Auburn Escape Zones: The Cabin in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Black Beard's Brig in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Imprisoned in Auburn, Alabama.
Made soup for the Birmingham, Alabama page. Sleeping for 1 seconds.
Saved URL for Breakout Games: The Kidnapping in Birmingham, Alabama.
Saved URL for Breakout Games: Runaway Train in Birmingham, Alabama.
Saved URL for Breakout Games: Museu

Made soup for the Orange Beach, Alabama page. Sleeping for 12 seconds.
Saved URL for Breakout Games: The Kidnapping in Orange Beach, Alabama.
Saved URL for Breakout Games: Hostage in Orange Beach, Alabama.
Saved URL for Breakout Games: Operation: Casino in Orange Beach, Alabama.
Saved URL for Breakout Games: Island Escape in Orange Beach, Alabama.
Saved URL for Breakout Games: Runaway Train in Orange Beach, Alabama.
Saved URL for Gulf Coast Escape Room:  Bates Motel in Orange Beach, Alabama.
Saved URL for XIT Escape Room: CSI: Gulf Shores in Orange Beach, Alabama.
Saved URL for XIT Escape Room: Movie Night in Orange Beach, Alabama.
Saved URL for Breakout Games: Runaway Train in Orange Beach, Alabama.
Saved URL for Breakout Games: Island Escape in Orange Beach, Alabama.
Saved URL for Escape House Waterville: The Hurricane Room in Orange Beach, Alabama.
Saved URL for Gulf Coast Escape Room: The Doll Room in Orange Beach, Alabama.
Saved URL for Get a Clue: Crime Scene in Orange Beach, Ala

Made soup for the Prescott, Arizona page. Sleeping for 3 seconds.
Saved URL for Prescott Escape Room: Dracula Experience in Prescott, Arizona.
Saved URL for Prescott Escape Room: Dracula Experience in Prescott, Arizona.
Made soup for the Tucson, Arizona page. Sleeping for 13 seconds.
Saved URL for Escape Room Tucson: Area 53: Lunar Lockdown in Tucson, Arizona.
Saved URL for Down the Rabbit Hole: The Asylum in Tucson, Arizona.
Saved URL for Ace of Escape: International Thief in Tucson, Arizona.
Saved URL for Escape Room Tucson: Red Rum Cottage in Tucson, Arizona.
Saved URL for Fox in a Box: Bank Robbery in Tucson, Arizona.
Saved URL for Will You Escape?: Serial Killer in Tucson, Arizona.
Saved URL for Down the Rabbit Hole: The Rabbit Hole: Welcome to 221B! in Tucson, Arizona.
Saved URL for Escape Room Tucson: Inside Job in Tucson, Arizona.
Saved URL for Ace of Escape: Bad Medicine in Tucson, Arizona.
Saved URL for Ace of Escape: Beat the Clock in Tucson, Arizona.
Saved URL for Will You 

Saved URL for Escape Hot Springs: Cold Case in Hot Springs, Arkansas.
Saved URL for A Narrow Escape: The Bionic Dilemma in Hot Springs, Arkansas.
Saved URL for A Narrow Escape: A Clinical Trial in Hot Springs, Arkansas.
Saved URL for Escape Hot Springs: The Bathhouse & Gambler in Hot Springs, Arkansas.
Saved URL for A Narrow Escape: Wonderland in Hot Springs, Arkansas.
Saved URL for Escape Hot Springs: Discoveries Through Time in Hot Springs, Arkansas.
Saved URL for A Narrow Escape: POD in Hot Springs, Arkansas.
Saved URL for A Narrow Escape: Murder at Silver City in Hot Springs, Arkansas.
Saved URL for A Narrow Escape: POD in Hot Springs, Arkansas.
Saved URL for A Narrow Escape: Murder at Silver City in Hot Springs, Arkansas.
Saved URL for Escape Hot Springs: Discoveries Through Time in Hot Springs, Arkansas.
Saved URL for Escape Hot Springs: The Bathhouse & Gambler in Hot Springs, Arkansas.
Saved URL for Escape Hot Springs: Cold Case in Hot Springs, Arkansas.
Saved URL for A Narrow E

Made soup for the Big Bear City, California page. Sleeping for 12 seconds.
Saved URL for Mountain Room Escapes: Mountain Ski Lodge in Big Bear City, California.
Saved URL for Mountain Room Escapes: The Cabin in Big Bear City, California.
Saved URL for Big Bear Escape Room: Bear Valley Search and Rescue in Big Bear City, California.
Saved URL for Big Bear Escape Room: Holcomb's Lost Gold in Big Bear City, California.
Saved URL for Mountain Room Escapes: The Inventor's Workshop in Big Bear City, California.
Saved URL for Big Bear Escape Room: Bear Valley Search and Rescue in Big Bear City, California.
Saved URL for Big Bear Escape Room: Holcomb's Lost Gold in Big Bear City, California.
Saved URL for Mountain Room Escapes: The Inventor's Workshop in Big Bear City, California.
Saved URL for Mountain Room Escapes: Mountain Ski Lodge in Big Bear City, California.
Saved URL for Mountain Room Escapes: The Cabin in Big Bear City, California.
Made soup for the Carlsbad, California page. Sleeping

...

Made soup for the Spokane, Washington page. Sleeping for 15 seconds.
Saved URL for Escape! Spokane: Prison Break: Warden Strikes Back! in Spokane, Washington.
Saved URL for Escape! Spokane: Intergalactic Express in Spokane, Washington.
Saved URL for Escape! Spokane: Saloon Shakedown in Spokane, Washington.
Saved URL for Escape! Spokane: High Noon Heist in Spokane, Washington.
Saved URL for Key Quest: Vacation Vandals in Spokane, Washington.
Saved URL for Key Quest: Detention in Spokane, Washington.
Saved URL for Escape! Spokane: Soviet Spy Game in Spokane, Washington.
Saved URL for Think Tank Escape Room: Mayan Doomsday in Spokane, Washington.
Saved URL for Escape! Spokane: Search and Rescue in Spokane, Washington.
Saved URL for Escape! Spokane: Secret Agent: Stealth Force in Spokane, Washington.
Saved URL for Escape! Spokane: The Fallout Shelter in Spokane, Washington.
Saved URL for Escape! Spokane: The Godfather's Secret in Spokane, Washington.
Saved URL for Escape! Spokane: Summonin

Made soup for the Parkersburg, West Virginia page. Sleeping for 11 seconds.
Made soup for the Princeton (WV), West Virginia page. Sleeping for 1 seconds.
Saved URL for Out of the Box Room Escape: Captain's Cabin in Princeton (WV), West Virginia.
Saved URL for Out of the Box Room Escape: Vanishing Act in Princeton (WV), West Virginia.
Saved URL for Out of the Box Room Escape: Last Option in Princeton (WV), West Virginia.
Saved URL for Out of the Box Room Escape: The Golden Lists in Princeton (WV), West Virginia.
Saved URL for Out of the Box Room Escape: Last Option in Princeton (WV), West Virginia.
Saved URL for Out of the Box Room Escape: The Golden Lists in Princeton (WV), West Virginia.
Saved URL for Out of the Box Room Escape: Vanishing Act in Princeton (WV), West Virginia.
Saved URL for Out of the Box Room Escape: Captain's Cabin in Princeton (WV), West Virginia.
Made soup for the Weirton, West Virginia page. Sleeping for 16 seconds.
Saved URL for EscapeWorks: Who Killed Bugsy Marl

Made soup for the Manitowoc, Wisconsin page. Sleeping for 19 seconds.
Saved URL for What The Lock?: Cold Case Files: The Bedroom in Manitowoc, Wisconsin.
Saved URL for What The Lock?: Outlaw: Jailbreak in Manitowoc, Wisconsin.
Saved URL for What The Lock?: Orvellio's: Curiosities and Conjury in Manitowoc, Wisconsin.
Saved URL for What The Lock?: Vlad: The Third Letter in Manitowoc, Wisconsin.
Saved URL for What The Lock?: Vlad: The Third Letter in Manitowoc, Wisconsin.
Saved URL for What The Lock?: Orvellio's: Curiosities and Conjury in Manitowoc, Wisconsin.
Saved URL for What The Lock?: Outlaw: Jailbreak in Manitowoc, Wisconsin.
Saved URL for What The Lock?: Cold Case Files: The Bedroom in Manitowoc, Wisconsin.
Made soup for the Milwaukee, Wisconsin page. Sleeping for 11 seconds.
Saved URL for Breakout Games: Mystery Mansion in Milwaukee, Wisconsin.
Saved URL for Breakout Games: The Kidnapping in Milwaukee, Wisconsin.
Saved URL for Breakout Games: Museum Heist in Milwaukee, Wisconsin.

Made soup for the Cheyenne, Wyoming page. Sleeping for 16 seconds.
Saved URL for Epic Escape Game: Myth of the West in Cheyenne, Wyoming.
Saved URL for Epic Escape Game: Curse of the Undead in Cheyenne, Wyoming.
Saved URL for Epic Escape Game: Clued In in Cheyenne, Wyoming.
Saved URL for Epic Escape Game: Unsolved Case in Cheyenne, Wyoming.
Saved URL for Epic Escape Game: Clued In in Cheyenne, Wyoming.
Saved URL for Epic Escape Game: Curse of the Undead in Cheyenne, Wyoming.
Saved URL for Epic Escape Game: Myth of the West in Cheyenne, Wyoming.
Saved URL for Epic Escape Game: Unsolved Case in Cheyenne, Wyoming.
Made soup for the Cody, Wyoming page. Sleeping for 11 seconds.
Made soup for the Evanston, Wyoming page. Sleeping for 5 seconds.
Saved URL for Room Escape Adventures: Back to the 90's in Evanston, Wyoming.
Saved URL for The Escape Game: Prison Break in Evanston, Wyoming.
Saved URL for The Escape Game: Mission: Mars in Evanston, Wyoming.
Saved URL for The Escape Game: Gold Rush i

Made soup for the Gillette, Wyoming page. Sleeping for 7 seconds.
Saved URL for 307 Mysteries: Justice Is Blind in Gillette, Wyoming.
Saved URL for 307 Mysteries: Commie Trickery in Gillette, Wyoming.
Saved URL for 307 Mysteries: Wacked Science in Gillette, Wyoming.
Saved URL for 307 Mysteries: Lifting The Jewels in Gillette, Wyoming.
Saved URL for 307 Mysteries: Miss Molly's Zombie Reboot in Gillette, Wyoming.
Saved URL for 307 Mysteries: Commie Trickery in Gillette, Wyoming.
Saved URL for 307 Mysteries: Miss Molly's Zombie Reboot in Gillette, Wyoming.
Saved URL for 307 Mysteries: Lifting The Jewels in Gillette, Wyoming.
Saved URL for 307 Mysteries: Justice Is Blind in Gillette, Wyoming.
Saved URL for 307 Mysteries: Wacked Science in Gillette, Wyoming.
Made soup for the Rock Springs, Wyoming page. Sleeping for 9 seconds.
Made soup for the Sheridan, Wyoming page. Sleeping for 5 seconds.
Saved URL for Escapade: Mystic Mistake in Sheridan, Wyoming.
Saved URL for Escapade: Sherlock and th

### Add Washington, D.C.

In [11]:
# add Washington, D.C. "state" container
state_city_room_urls["Washington, D.C."] = {}

# add Washington, D.C. "city" container
state_city_room_urls["Washington, D.C."]["Washington, D.C."] = {}

# soup
wdc_soup = BeautifulSoup(requests.get("https://worldofescapes.com/washington").text, "html.parser")

# list containers for escape room company name, escape room game name, and url
company_name = []
room_name = []
room_url = []
        
# company title
for company_link in wdc_soup.find_all("a", {"class": "company-link"}):
    company_name.append(str(re.findall('title="(.*?)"', str(company_link))).strip("['']"))
                
# game title
for room_link in wdc_soup.find_all("a", {"class": "quest_tile_name_link"}):
    room_name.append(room_link.get_text())
                    
# concatenate company name: room name
company_and_room = [": ".join([company, room]) for company, room in zip(company_name, room_name)]
                
# add lists for company-room labels
for company_and_room_k in company_and_room:
    state_city_room_urls["Washington, D.C."]["Washington, D.C."][company_and_room_k] = []
                
# loop through href for each room, append to home page to make url
for a in wdc_soup.find("section", {"class": "container"}).find_all("a", {"class": "item-hover quest_tile_hover_link"}):
    room_url.append(home_page_url + a["href"])
                
# append ...
for company_and_room_k, room_url_k in zip(company_and_room, room_url):
    state_city_room_urls["Washington, D.C."]["Washington, D.C."][company_and_room_k].append(room_url_k)
                
    # print loop result
    print("Saved URL for {} in {}, {}.".format(company_and_room_k, "Washington, D.C.", "Washington, D.C."))

# delete soup object
del wdc_soup

Saved URL for Breakout Games: Mystery Mansion in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Hostage in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Operation: Casino in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: The Kidnapping in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Runaway Train in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Island Escape in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Undercover Alley in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Undercover Alley in Washington, D.C., Washington, D.C..
Saved URL for Escape Room Live: Odditorium Emporium Shopping Spree in Washington, D.C., Washington, D.C..
Saved URL for Exit Plan: Refuge in Washington, D.C., Washington, D.C..
Saved URL for Room Escape DC: Magic School for Kids in Washington, D.C., Washington, D.C..
Saved URL for Escapology: Antidote in Washington, D.C., Washi

### Print URLs

In [12]:
state_city_room_urls

{'Alabama': {'Auburn': {'Auburn Escape Zones: The Cabin': ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-cabin',
    'https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-cabin'],
   'Auburn Escape Zones: Imprisoned': ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-imprisoned',
    'https://worldofescapes.com/auburn/quests/auburn-escape-zones-imprisoned'],
   'Auburn Escape Zones: Vault': ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-vault',
    'https://worldofescapes.com/auburn/quests/auburn-escape-zones-vault'],
   'Auburn Escape Zones: The Puzzler vs Superheroes': ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-puzzler-vs-superheroes',
    'https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-puzzler-vs-superheroes'],
   "Auburn Escape Zones: Black Beard's Brig": ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-black-beards-brig',
    'https://worldofescapes.com/auburn/quest

### Store room urls in a data frame

In [13]:
# data frame container
state_city_room_url_df = pd.DataFrame(columns = ["state", "city", "company_and_room", "woe_room_url"])

# loop through city values nested in state keys
for state_key, city_value in state_city_room_urls.items():
    # loop through room values nested within city keys
    for city_key, room_value in city_value.items():
        # loop through urls nested in room keys
        for room_key, url_value in room_value.items():
            # loop through urls (some duplicates on the page)
            for url_key, url in enumerate(url_value):
                # append to data frame (append does not occur in place, so need to assign appended data frame)
                state_city_room_url_df = state_city_room_url_df.append(pd.DataFrame({"state": state_key, "city": city_key, "company_and_room": room_key, "woe_room_url": url}, index = [0]), ignore_index = True)

# See it
state_city_room_url_df

,state,city,company_and_room,woe_room_url
0,Alabama,Auburn,Auburn Escape Zones: The Cabin,https://worldofescapes.com/auburn/quests/aubur...
1,Alabama,Auburn,Auburn Escape Zones: The Cabin,https://worldofescapes.com/auburn/quests/aubur...
2,Alabama,Auburn,Auburn Escape Zones: Imprisoned,https://worldofescapes.com/auburn/quests/aubur...
3,Alabama,Auburn,Auburn Escape Zones: Imprisoned,https://worldofescapes.com/auburn/quests/aubur...
4,Alabama,Auburn,Auburn Escape Zones: Vault,https://worldofescapes.com/auburn/quests/aubur...
...,...,...,...,...
10308,"Washington, D.C.","Washington, D.C.",Escape Quest: The King’s Ransom,https://worldofescapes.com/washington/quests/e...
10309,"Washington, D.C.","Washington, D.C.",PanIQ Escape Room: Primal Quest,https://worldofescapes.com/washington/quests/p...
10310,"Washington, D.C.","Washington, D.C.",Escape Room Live: Ghostbusters!,https://worldofescapes.com/washington/quests/e...
10311,"Washington, D.C.","Washington, D.C.",Escape Room Live: Friday the 13th,https://worldofescapes.com/washington/quests/e...


### Drop duplicates and write .csv

In [15]:
# drop duplicatesa
state_city_room_url_df = state_city_room_url_df.drop_duplicates(inplace = False)

# write to .csv
state_city_room_url_df.to_csv("data/state_city_room_url_df.csv", index = False)

## Example page soup: Weird Heritage by Room 5280 in Denver, CO

In [16]:
room_soup_example = BeautifulSoup(requests.get("https://worldofescapes.com/denver/quests/room-5280-weird-heritage").text)

## Define functions for extracting information from room soup

### Room/Game Title

In [17]:
def get_title(soup):
    try:
        return soup.find("title").get_text()
    except:
        return "None"

# test
get_title(room_soup_example)

'Escape room "Weird Heritage" by ROOM 5280 in Denver'

### Room/Game Description

In [18]:
def get_description(soup):
    try:
        return re.sub("Description:", "", soup.find("div", {"class": "description"}).get_text())
    except:
        return "None"

# test
get_description(room_soup_example)

'This is for real! You will find yourself in a strange apartment which you were lucky to inherit from a mysterious relative of yours. How cool is that? The only problem is that you have never heard of this “relative” before and what’s more important, now that the door is locked, you do not know how to escape!'

### Room/Game Address

In [19]:
def get_address(soup):
    try:
        return re.sub(" \(Show on map\)", "", soup.find("div", {"data-content": "address"}).get_text().strip())
    except:
        return "None"

# test
get_address(room_soup_example)

'142 W 5th Ave, Denver, CO 80204'

### Room/Game Tags

In [20]:
def get_tags(soup):
    try:
        return [li.get_text() for li in soup.find("div", {"class": "tags"}).find("ul", {"class": "tags-2"}).find_all("li")]
    except:
        return ["None"]

# test
get_tags(room_soup_example)

['Public Ticketing', 'Up to ten players']

### Room/Game User Reviews

In [21]:
def get_reviews(soup):
    try:
        return [content.get_text() for content in soup.find("ul", {"class": "masonry-list"}).find_all("p", {"class": "content"})]
    except:
        return ["None"]

# test
get_reviews(room_soup_example)

['This place was awesome! For many in our group, this was our first escape room and it was the perfect amount of challenge. The room we did (Weird Heritage) was extremely well thought out and the puzzles were fun and interactive. The host was funny and gave us a hint only when we really needed it.',
 'We had a GREAT time in the Heritage room! There were six of us, including my nine-year-old daughter, and really I think you need a bare minimum of four to make this work, but the owner said you really need six. The owner is very hospitable and the clues are challenging but not impossible.We escaped with eight minutes to spare and were pretty happy about it!  Highly recommend.',
 'We did "Weird Heritage," and it was great! They had a good variety of puzzles and challenges. Can\'t wait to try another escape room here!',
 'Had a great time! Escaped The Heritage room. Greet for a larger group! Thanks so much!']

### Room/Game Player Range

In [22]:
def get_player_range(soup):
    try:
        return re.sub("\u200a–\u200a", "-", soup.find("ul", {"class": "params-ul"}).find("li", {"data-content": "participants-count"}).get_text().strip("Number of players"))
    except:
        return "None"

# test
get_player_range(room_soup_example)

'4-10'

### Room/Game Time Limit

In [23]:
def get_time_limit(soup):
    try:
        return int(soup.find("ul", {"class": "params-ul"}).find("li", {"data-content": "time"}).get_text().split()[1].split("limit")[1])
    except:
        return "None"

# test
get_time_limit(room_soup_example)

60

### Room/Game Difficulty Level

In [24]:
def get_difficulty_level(soup):
    try:
        for tooltip in soup.find("ul", {"class": "params-ul"}).find_all("span", {"data-toggle": "tooltip"}):
            # string match easy or difficult
            if "easy" or "difficult" in str(tooltip).lower():
                return str(re.findall('data-original-title="(.*?)"', str(tooltip))).strip("['']")
    except:
        return "None"

# test
get_difficulty_level(room_soup_example)

'Difficult'

### Room/Game Fear Level

In [25]:
def get_fear_level(soup):
    try:
        if re.findall('data-original-title="(.*?)"', str(soup.find("ul", {"class": "params-ul"}).find("span", {"class": "td scary"}))) != []:
            return str(re.findall('data-original-title="(.*?)"', str(soup.find("ul", {"class": "params-ul"}).find("span", {"class": "td scary"})))).strip("['']")
        else:
            return soup.find("ul", {"class": "params-ul"}).find("span", {"class": "td scary"}).get_text()
    except:
        return "None"

# test
get_fear_level(room_soup_example)

'Not scary'

### Room/Game Age Requirement

In [26]:
def get_age_requirement(soup):
    try:
        for tooltip in soup.find("ul", {"class": "params-ul"}).find_all("span", {"data-toggle": "tooltip"}):
            # age requirement includes a "+"
            if "+" in str(tooltip):
                return str(re.findall(">(.*?)<", str(tooltip))[0]).strip("['']")
    except:
        return "None"

# test
get_age_requirement(room_soup_example)

'12+'

### Room/Game Success Rate

In [27]:
def get_success_rate(soup):
    try:
        return float(soup.find("span", {"data-original-title": "Success rate:"}).get_text().strip(" %")) / 100
    except:
        return "None"

# test
get_success_rate(room_soup_example)

0.3

## Combine functions into one

### Most room data except reviews and tags

In [29]:
def get_room_data(url, soup):
    return pd.DataFrame({"room_title": get_title(soup), "woe_room_url": url, "room_address": get_address(soup), "room_description": get_description(soup), "player_range": get_player_range(soup), "time_limit": get_time_limit(soup), "difficulty_level": get_difficulty_level(soup), "fear_level": get_fear_level(soup), "age_requirement": get_age_requirement(soup), "success_rate": get_success_rate(soup)}, index = [0])

# test
get_room_data("https://worldofescapes.com/denver/quests/room-5280-weird-heritage", room_soup_example)

,room_title,woe_room_url,room_address,room_description,player_range,time_limit,difficulty_level,fear_level,age_requirement,success_rate
0,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,"142 W 5th Ave, Denver, CO 80204",This is for real! You will find yourself in a ...,4-10,60,Difficult,Not scary,12+,0.3


### Review data

In [30]:
def get_review_data(url, soup):
    # room title and reviews
    room_title = get_title(soup)
    reviews = get_reviews(soup)
    
    # data frame container
    data_frame = pd.DataFrame(columns = ["room_title", "woe_room_url", "review_id", "review"])
    
    # loop through reviews
    for i, review_i in enumerate(reviews):
        data_frame = data_frame.append(pd.DataFrame({"room_title": room_title, "woe_room_url": url, "review_id": i, "review": review_i}, index = [0]), ignore_index = True)
    
    return data_frame

# test
get_review_data("https://worldofescapes.com/denver/quests/room-5280-weird-heritage", room_soup_example)

,room_title,woe_room_url,review_id,review
0,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,0,"This place was awesome! For many in our group,..."
1,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,1,We had a GREAT time in the Heritage room! Ther...
2,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,2,"We did ""Weird Heritage,"" and it was great! The..."
3,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,3,Had a great time! Escaped The Heritage room. G...


### Tags data

In [31]:
def get_tag_data(url, soup):
    # room title and reviews
    room_title = get_title(soup)
    tags = get_tags(soup)
    
    # data frame container
    data_frame = pd.DataFrame(columns = ["room_title", "woe_room_url", "tag_id", "tag"])
    
    # loop through reviews
    for i, tag_i in enumerate(tags):
        data_frame = data_frame.append(pd.DataFrame({"room_title": room_title, "woe_room_url": url, "tag_id": i, "tag": tag_i}, index = [0]), ignore_index = True)
    
    return data_frame

# test
get_tag_data("https://worldofescapes.com/denver/quests/room-5280-weird-heritage", room_soup_example)

,room_title,woe_room_url,tag_id,tag
0,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,0,Public Ticketing
1,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,1,Up to ten players


## Loop through room URLs, extract data, and write to .csv files

In [32]:
# read most recent data
state_city_room_url_df = pd.read_csv("data/state_city_room_url_df.csv")

for url in state_city_room_url_df["woe_room_url"]:
    # href for naming .csv
    room_href = url.split("quests/")[1]
    
    # soup
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    
    # write room data to .csv
    get_room_data(url, soup).to_csv("data/room_data/" + room_href + "_room_data.csv", index = False)
    
    # write review data to .csv
    get_review_data(url, soup).to_csv("data/reviews/" + room_href + "_room_reviews.csv", index = False)
    
    # write tag data to .csv
    get_tag_data(url, soup).to_csv("data/tags/" + room_href + "_room_tags.csv", index = False)
        
    # sleep for between 1-20 seconds
    sleep_time_i = np.random.randint(low = 1, high = 20, size = 1)
    
    # print iteration results
    print("Exported .csv files for {}. Waiting {} seconds before making more soup.".format(room_href, str(sleep_time_i).strip("[]")))
    
    # now sleep
    time.sleep(sleep_time_i)

Exported .csv files for auburn-escape-zones-the-cabin. Waiting 9 seconds before making more soup.
Exported .csv files for auburn-escape-zones-imprisoned. Waiting 18 seconds before making more soup.
Exported .csv files for auburn-escape-zones-vault. Waiting 19 seconds before making more soup.
Exported .csv files for auburn-escape-zones-the-puzzler-vs-superheroes. Waiting 5 seconds before making more soup.
Exported .csv files for auburn-escape-zones-black-beards-brig. Waiting 6 seconds before making more soup.
Exported .csv files for Breakout-Birmingham-The-Kidnapping. Waiting 3 seconds before making more soup.
Exported .csv files for breakout-games-birmingham-runaway-train. Waiting 5 seconds before making more soup.
Exported .csv files for Breakout-Birmingham-The-Musuem-Heist. Waiting 13 seconds before making more soup.
Exported .csv files for breakout-games-birmingham-do-not-disturb. Waiting 1 seconds before making more soup.
Exported .csv files for breakout-games-atlanta-mystery-mansi

Exported .csv files for xit-escape-room-movie-night. Waiting 7 seconds before making more soup.
Exported .csv files for escape-house-waterville-the-hurricane-room. Waiting 9 seconds before making more soup.
Exported .csv files for Gulf-Coast-Escape-Room-THE-DOLL-ROOM. Waiting 9 seconds before making more soup.
Exported .csv files for get-a-clue-crime-scene. Waiting 3 seconds before making more soup.
Exported .csv files for get-a-clue-mardi-gras. Waiting 8 seconds before making more soup.
Exported .csv files for xit-escape-room-insane-asylum. Waiting 6 seconds before making more soup.
Exported .csv files for escape-house-waterville-the-room-of-the-great-mysto. Waiting 4 seconds before making more soup.
Exported .csv files for Gulf-Coast-Escape-Room-ZOMBIE-LAB. Waiting 2 seconds before making more soup.
Exported .csv files for get-a-clue-paranormal-investigation. Waiting 12 seconds before making more soup.
Exported .csv files for xit-escape-room-a-tale-of-two-forts. Waiting 14 seconds be

Exported .csv files for dare-to-escape-the-cult-of-mithras. Waiting 18 seconds before making more soup.
Exported .csv files for alcatraz-escape-games-zombie-panic. Waiting 2 seconds before making more soup.
Exported .csv files for escape-westgate-the-lost-tomb. Waiting 18 seconds before making more soup.
Exported .csv files for escape-rooms-mesa-outbreak. Waiting 14 seconds before making more soup.
Exported .csv files for cluemaster-escape-rooms-the-cell-block. Waiting 7 seconds before making more soup.
Exported .csv files for escape-house-mesa-flood-the-city. Waiting 11 seconds before making more soup.
Exported .csv files for phoenix-puzzle-room-the-treasure-of-captain-lockjaw. Waiting 11 seconds before making more soup.
Exported .csv files for eludesions-escape-rooms-case-the-wicked-witchs-castle. Waiting 17 seconds before making more soup.
Exported .csv files for eludesions-escape-rooms-yin-and-yang. Waiting 17 seconds before making more soup.
Exported .csv files for north-valley-es

Exported .csv files for escape-games-az-monkey-business. Waiting 9 seconds before making more soup.
Exported .csv files for paradox-escape-room-caskwerks-and-capone. Waiting 7 seconds before making more soup.
Exported .csv files for escape-rooms-mesa-suspects. Waiting 14 seconds before making more soup.
Exported .csv files for north-valley-escape-room-big-top-circus. Waiting 16 seconds before making more soup.
Exported .csv files for paradox-escape-room-the-collector. Waiting 3 seconds before making more soup.
Exported .csv files for international-room-escape-az-gravitational-effect. Waiting 1 seconds before making more soup.
Exported .csv files for escape-room-101-space-station-tiberia. Waiting 4 seconds before making more soup.
Exported .csv files for mindspark-escape-games-terminal. Waiting 16 seconds before making more soup.
Exported .csv files for escape-rooms-mesa-disco-fever. Waiting 12 seconds before making more soup.
Exported .csv files for the-dig-az. Waiting 17 seconds befor

Exported .csv files for great-escape-mystery-rooms-prison-break. Waiting 10 seconds before making more soup.
Exported .csv files for fort-smith-escape-room-the-great-cookie-catastrophe. Waiting 7 seconds before making more soup.
Exported .csv files for fort-smith-escape-room-the-missing. Waiting 11 seconds before making more soup.
Exported .csv files for fort-smith-escape-room-blood-on-bayou. Waiting 16 seconds before making more soup.
Exported .csv files for fort-smith-escape-room-nightmare-hotel. Waiting 14 seconds before making more soup.
Exported .csv files for great-escape-mystery-rooms-fbi-academy. Waiting 15 seconds before making more soup.
Exported .csv files for cluemasters-escape-room-the-game-show. Waiting 15 seconds before making more soup.
Exported .csv files for cluemasters-escape-room-the-missing-professor. Waiting 19 seconds before making more soup.
Exported .csv files for cluemasters-the-alchemy-lab. Waiting 16 seconds before making more soup.
Exported .csv files for g

...

Exported .csv files for what-the-lock-cold-case-files-the-bedroom. Waiting 15 seconds before making more soup.
Exported .csv files for what-the-lock-outlaw-jailbreak. Waiting 11 seconds before making more soup.
Exported .csv files for what-the-lock-orvellios-curiosities-and-conjury. Waiting 5 seconds before making more soup.
Exported .csv files for what-the-lock-vlad-the-third-letter. Waiting 13 seconds before making more soup.
Exported .csv files for breakout-games-milwaukee-mystery-mansion. Waiting 4 seconds before making more soup.
Exported .csv files for breakout-games-milwaukee-the-kidnapping. Waiting 10 seconds before making more soup.
Exported .csv files for breakout-games-milwaukee-museum-heist. Waiting 3 seconds before making more soup.
Exported .csv files for breakout-games-milwaukee-operation-casino. Waiting 2 seconds before making more soup.
Exported .csv files for breakout-games-milwaukee-hostage. Waiting 16 seconds before making more soup.
Exported .csv files for breakout

Exported .csv files for epic-escape-game-countdown-2-kick-off. Waiting 3 seconds before making more soup.
Exported .csv files for room-escape-adventures-ch-back-to-the-90s. Waiting 11 seconds before making more soup.
Exported .csv files for the-escape-game-chicago-prison-break. Waiting 8 seconds before making more soup.
Exported .csv files for the-escape-game-chicago-mission-mars. Waiting 17 seconds before making more soup.
Exported .csv files for the-escape-game-chicago-gold-rush. Waiting 3 seconds before making more soup.
Exported .csv files for the-escape-game-chicago-special-ops. Waiting 5 seconds before making more soup.
Exported .csv files for the-escape-game-chicago-the-heist. Waiting 3 seconds before making more soup.
Exported .csv files for the-escape-game-chicago-team-building-opportunities. Waiting 11 seconds before making more soup.
Exported .csv files for trapped-with-zombie-chicago. Waiting 5 seconds before making more soup.
Exported .csv files for the-escape-game-chicago

Exported .csv files for the-room-upstairs-rock-and-roll-medical-center. Waiting 9 seconds before making more soup.
Exported .csv files for mastermind-escape-games-il-sorcerers-secret. Waiting 16 seconds before making more soup.
Exported .csv files for challenge-accepted-the-office. Waiting 6 seconds before making more soup.
Exported .csv files for odyssey-escape-game-schaumburg-jack-the-ripper. Waiting 15 seconds before making more soup.
Exported .csv files for mind-trap-escape-rooms-escape-dr-death. Waiting 7 seconds before making more soup.
Exported .csv files for escape-the-room-chicago-the-submarine. Waiting 17 seconds before making more soup.
Exported .csv files for mind-trap-escape-rooms-pharaohs-chamber. Waiting 1 seconds before making more soup.
Exported .csv files for challenge-accepted-villains-lair. Waiting 3 seconds before making more soup.
Exported .csv files for escapology-orland-park-7-deadly-sins. Waiting 8 seconds before making more soup.
Exported .csv files for psych-

Exported .csv files for psych-escape-room-the-starlight-diner. Waiting 18 seconds before making more soup.
Exported .csv files for paniq-escape-room-chicago-wizard-trials. Waiting 2 seconds before making more soup.
Exported .csv files for panic-escape-room-hostel. Waiting 19 seconds before making more soup.
Exported .csv files for no-escape-room-gragon-tower. Waiting 18 seconds before making more soup.
Exported .csv files for no-escape-room-space-station-tiberia. Waiting 9 seconds before making more soup.
Exported .csv files for mault-scape-chicag-mad-scientist. Waiting 10 seconds before making more soup.
Exported .csv files for outatime-games-toy-rescue. Waiting 6 seconds before making more soup.
Exported .csv files for lock-chicago-malfunction. Waiting 19 seconds before making more soup.
Exported .csv files for the-great-escape-room-chicago-sherlock. Waiting 19 seconds before making more soup.
Exported .csv files for mind-trap-escape-rooms-escape-storm-surge. Waiting 10 seconds befor

Exported .csv files for Escape-the-Room-DC-The-Agency. Waiting 15 seconds before making more soup.
Exported .csv files for paniq-room-perfect-crime. Waiting 10 seconds before making more soup.
Exported .csv files for escape-room-live-curse-of-the-mummy. Waiting 12 seconds before making more soup.
Exported .csv files for Escape-Artist-DC-House-of-Pawns. Waiting 13 seconds before making more soup.
Exported .csv files for smart-escape-towson-the-ring. Waiting 9 seconds before making more soup.
Exported .csv files for insomnia-escape-room-dc-dungeon-things. Waiting 6 seconds before making more soup.
Exported .csv files for smart-escape-towson-the-golden-bug. Waiting 3 seconds before making more soup.
Exported .csv files for escape-room-herndon-8-bit-escape. Waiting 2 seconds before making more soup.
Exported .csv files for room-escape-dc-bowl-voyage. Waiting 8 seconds before making more soup.
Exported .csv files for smart-escape-towson-bank-robbery. Waiting 11 seconds before making more so

### Read room data .csv files and concatenate together

In [33]:
room_data = pd.concat([pd.read_csv(filepath_i) for filepath_i in glob.glob("data/room_data/*.csv")], ignore_index = True)

# See it
room_data

,room_title,woe_room_url,room_address,room_description,player_range,time_limit,difficulty_level,fear_level,age_requirement,success_rate
0,"Escape room ""The Pirate's Curse"" by Escape Roo...",https://worldofescapes.com/washington/quests/e...,"3345 M Street NW, Washington, DC 20007","Ahoy, Matey! All aboard Ye Pirate Ship! All wh...",8-10,60,Very difficult,Not scary,5+,None
1,"Escape room ""The Time Machine"" by MindTrix in ...",https://worldofescapes.com/portland/quests/mid...,"12000 SE 82nd Ave #1009 Happy Valley, OR 97086",Scientists are on the verge of discovering tim...,2-8,60,Difficult,Not scary,14+,None
2,"Escape room ""National Treasure"" by The Escape ...",https://worldofescapes.com/new-brunswick/quest...,"45 Lafayette Rd Fords, NJ 08863",While on vacation with a group of your buddies...,2-10,60,Very difficult,Not scary,NaN,None
3,"Escape room ""The Lost Antidote"" by The Escape ...",https://worldofescapes.com/little-rock/quests/...,"1214 S Main St Little Rock, AR 72202","Dr. Stapleton, an evil scientist, has poisoned...",2-7,60,Average,Not scary,13+,0.4
4,"Escape room ""Outbreak: Find the Cure"" by Maste...",https://worldofescapes.com/overland-park/quest...,"7810 West 151st Street Overland Park, KS 66223",The CDC is on lockdown after a deadly virus wa...,2-10,60,Difficult,Not scary,NaN,0.21
...,...,...,...,...,...,...,...,...,...,...
7447,"Escape room ""Nuclear Shuttle"" by Escape Rooms ...",https://worldofescapes.com/dallas/quests/escap...,"1565 W Main ST #495 Lewisville, TX 75077",Your team is the second shift crew for the Orb...,2-8,60,Difficult,Not scary,13+,0.3
7448,"Escape room ""The Wizard's Chamber"" by Escape N...",https://worldofescapes.com/warren-oh/quests/es...,"200 Chestnut Avenue Northeast Suite C Warren, ...",The founders created a vessel to tether their ...,4-8,60,Difficult,Not scary,10+,None
7449,"Escape room ""Houdini's Magic Cell - Destinatio...",https://worldofescapes.com/andover/quests/all-...,"301 Mt Hope Ave Rockaway, NJ 07866","Just like Houdini began, you will start your q...",2-10,60,Difficult,Not scary,14+,None
7450,"Escape room ""High Noon in the Old West"" by Tim...",https://worldofescapes.com/atlanta/quests/time...,"1441 Dresden Dr Suite 230 Atlanta, GA 30319",It is 1899 and the final push to tame the wild...,1-10,60,Average,Not scary,12+,None


#### Recode room variables

In [5]:
# modify variables
room_data = room_data.assign(min_players = room_data["player_range"].apply(lambda x: x[:1] if x != "None" else "None"),
                             max_players = room_data["player_range"].apply(lambda x: x.split("-")[1] if x != "None" else "None"),
                             time_limit_str = room_data["time_limit"].apply(lambda x: str(x) + " minutes" if x != "None" else "None"),
                             minimum_age = room_data["age_requirement"].apply(lambda x: code_minimum_age(x)),
                             difficulty_int = room_data["difficulty_level"].apply(lambda x: recode_difficulty(x)),
                             fear_int = room_data["fear_level"].apply(lambda x: recode_fear(x)))

TypeError: 'float' object is not subscriptable

#### Save latitude and longitude as lists

In [38]:
# query address, latitude, and longitude columns
# query_address, latitude, longitude = [], [], []
for i, address in enumerate(room_data["room_address"][3449:]):
    # get
    try:
        address_lat_long = get_lat_long(address)
    
        # append
        query_address.append(address_lat_long[0])
        latitude.append(address_lat_long[1])
        longitude.append(address_lat_long[2])
    
        # iteration complete message
        print("{} out of {} addresses completed.".format(i, len(room_data["room_address"])))
    
        # sleep for 1 seconds
        time.sleep(1)
    except:
        # append
        query_address.append("None")
        latitude.append("None")
        longitude.append("None")
    
        # iteration complete message
        print("NONE. {} out of {} addresses completed.".format(i, len(room_data["room_address"])))
    
        # sleep for 1 seconds
        time.sleep(1)

NONE. 0 out of 7452 addresses completed.
1 out of 7452 addresses completed.
2 out of 7452 addresses completed.
3 out of 7452 addresses completed.
4 out of 7452 addresses completed.
5 out of 7452 addresses completed.
6 out of 7452 addresses completed.
7 out of 7452 addresses completed.
8 out of 7452 addresses completed.
9 out of 7452 addresses completed.
10 out of 7452 addresses completed.
11 out of 7452 addresses completed.
12 out of 7452 addresses completed.
13 out of 7452 addresses completed.
14 out of 7452 addresses completed.
15 out of 7452 addresses completed.
16 out of 7452 addresses completed.
17 out of 7452 addresses completed.
18 out of 7452 addresses completed.
19 out of 7452 addresses completed.
20 out of 7452 addresses completed.
21 out of 7452 addresses completed.
22 out of 7452 addresses completed.
23 out of 7452 addresses completed.
24 out of 7452 addresses completed.
25 out of 7452 addresses completed.
26 out of 7452 addresses completed.
27 out of 7452 addresses complet

225 out of 7452 addresses completed.
226 out of 7452 addresses completed.
227 out of 7452 addresses completed.
228 out of 7452 addresses completed.
229 out of 7452 addresses completed.
230 out of 7452 addresses completed.
231 out of 7452 addresses completed.
232 out of 7452 addresses completed.
233 out of 7452 addresses completed.
234 out of 7452 addresses completed.
235 out of 7452 addresses completed.
236 out of 7452 addresses completed.
237 out of 7452 addresses completed.
238 out of 7452 addresses completed.
239 out of 7452 addresses completed.
240 out of 7452 addresses completed.
241 out of 7452 addresses completed.
242 out of 7452 addresses completed.
243 out of 7452 addresses completed.
244 out of 7452 addresses completed.
245 out of 7452 addresses completed.
246 out of 7452 addresses completed.
247 out of 7452 addresses completed.
248 out of 7452 addresses completed.
249 out of 7452 addresses completed.
250 out of 7452 addresses completed.
251 out of 7452 addresses completed.
2

447 out of 7452 addresses completed.
448 out of 7452 addresses completed.
449 out of 7452 addresses completed.
450 out of 7452 addresses completed.
451 out of 7452 addresses completed.
452 out of 7452 addresses completed.
453 out of 7452 addresses completed.
454 out of 7452 addresses completed.
455 out of 7452 addresses completed.
456 out of 7452 addresses completed.
457 out of 7452 addresses completed.
458 out of 7452 addresses completed.
459 out of 7452 addresses completed.
460 out of 7452 addresses completed.
461 out of 7452 addresses completed.
462 out of 7452 addresses completed.
463 out of 7452 addresses completed.
464 out of 7452 addresses completed.
465 out of 7452 addresses completed.
466 out of 7452 addresses completed.
467 out of 7452 addresses completed.
468 out of 7452 addresses completed.
469 out of 7452 addresses completed.
470 out of 7452 addresses completed.
471 out of 7452 addresses completed.
472 out of 7452 addresses completed.
473 out of 7452 addresses completed.
4

669 out of 7452 addresses completed.
670 out of 7452 addresses completed.
671 out of 7452 addresses completed.
672 out of 7452 addresses completed.
673 out of 7452 addresses completed.
674 out of 7452 addresses completed.
675 out of 7452 addresses completed.
676 out of 7452 addresses completed.
677 out of 7452 addresses completed.
678 out of 7452 addresses completed.
679 out of 7452 addresses completed.
680 out of 7452 addresses completed.
681 out of 7452 addresses completed.
682 out of 7452 addresses completed.
683 out of 7452 addresses completed.
684 out of 7452 addresses completed.
685 out of 7452 addresses completed.
686 out of 7452 addresses completed.
687 out of 7452 addresses completed.
688 out of 7452 addresses completed.
689 out of 7452 addresses completed.
690 out of 7452 addresses completed.
691 out of 7452 addresses completed.
692 out of 7452 addresses completed.
693 out of 7452 addresses completed.
694 out of 7452 addresses completed.
695 out of 7452 addresses completed.
6

891 out of 7452 addresses completed.
892 out of 7452 addresses completed.
893 out of 7452 addresses completed.
894 out of 7452 addresses completed.
895 out of 7452 addresses completed.
896 out of 7452 addresses completed.
897 out of 7452 addresses completed.
898 out of 7452 addresses completed.
899 out of 7452 addresses completed.
900 out of 7452 addresses completed.
901 out of 7452 addresses completed.
902 out of 7452 addresses completed.
903 out of 7452 addresses completed.
904 out of 7452 addresses completed.
905 out of 7452 addresses completed.
906 out of 7452 addresses completed.
907 out of 7452 addresses completed.
908 out of 7452 addresses completed.
909 out of 7452 addresses completed.
910 out of 7452 addresses completed.
911 out of 7452 addresses completed.
912 out of 7452 addresses completed.
913 out of 7452 addresses completed.
914 out of 7452 addresses completed.
915 out of 7452 addresses completed.
916 out of 7452 addresses completed.
917 out of 7452 addresses completed.
9

...

3054 out of 7452 addresses completed.
3055 out of 7452 addresses completed.
3056 out of 7452 addresses completed.
3057 out of 7452 addresses completed.
3058 out of 7452 addresses completed.
3059 out of 7452 addresses completed.
3060 out of 7452 addresses completed.
3061 out of 7452 addresses completed.
3062 out of 7452 addresses completed.
3063 out of 7452 addresses completed.
3064 out of 7452 addresses completed.
3065 out of 7452 addresses completed.
3066 out of 7452 addresses completed.
3067 out of 7452 addresses completed.
3068 out of 7452 addresses completed.
3069 out of 7452 addresses completed.
3070 out of 7452 addresses completed.
3071 out of 7452 addresses completed.
3072 out of 7452 addresses completed.
3073 out of 7452 addresses completed.
3074 out of 7452 addresses completed.
3075 out of 7452 addresses completed.
3076 out of 7452 addresses completed.
3077 out of 7452 addresses completed.
3078 out of 7452 addresses completed.
3079 out of 7452 addresses completed.
3080 out of 

3270 out of 7452 addresses completed.
3271 out of 7452 addresses completed.
3272 out of 7452 addresses completed.
3273 out of 7452 addresses completed.
3274 out of 7452 addresses completed.
3275 out of 7452 addresses completed.
3276 out of 7452 addresses completed.
3277 out of 7452 addresses completed.
3278 out of 7452 addresses completed.
3279 out of 7452 addresses completed.
3280 out of 7452 addresses completed.
3281 out of 7452 addresses completed.
3282 out of 7452 addresses completed.
3283 out of 7452 addresses completed.
3284 out of 7452 addresses completed.
3285 out of 7452 addresses completed.
3286 out of 7452 addresses completed.
3287 out of 7452 addresses completed.
3288 out of 7452 addresses completed.
3289 out of 7452 addresses completed.
3290 out of 7452 addresses completed.
3291 out of 7452 addresses completed.
3292 out of 7452 addresses completed.
3293 out of 7452 addresses completed.
3294 out of 7452 addresses completed.
3295 out of 7452 addresses completed.
3296 out of 

3486 out of 7452 addresses completed.
3487 out of 7452 addresses completed.
3488 out of 7452 addresses completed.
3489 out of 7452 addresses completed.
3490 out of 7452 addresses completed.
3491 out of 7452 addresses completed.
3492 out of 7452 addresses completed.
3493 out of 7452 addresses completed.
3494 out of 7452 addresses completed.
3495 out of 7452 addresses completed.
3496 out of 7452 addresses completed.
3497 out of 7452 addresses completed.
3498 out of 7452 addresses completed.
3499 out of 7452 addresses completed.
3500 out of 7452 addresses completed.
3501 out of 7452 addresses completed.
3502 out of 7452 addresses completed.
3503 out of 7452 addresses completed.
3504 out of 7452 addresses completed.
3505 out of 7452 addresses completed.
3506 out of 7452 addresses completed.
3507 out of 7452 addresses completed.
3508 out of 7452 addresses completed.
3509 out of 7452 addresses completed.
3510 out of 7452 addresses completed.
3511 out of 7452 addresses completed.
3512 out of 

3702 out of 7452 addresses completed.
3703 out of 7452 addresses completed.
3704 out of 7452 addresses completed.
3705 out of 7452 addresses completed.
3706 out of 7452 addresses completed.
3707 out of 7452 addresses completed.
3708 out of 7452 addresses completed.
3709 out of 7452 addresses completed.
3710 out of 7452 addresses completed.
3711 out of 7452 addresses completed.
3712 out of 7452 addresses completed.
3713 out of 7452 addresses completed.
3714 out of 7452 addresses completed.
3715 out of 7452 addresses completed.
3716 out of 7452 addresses completed.
3717 out of 7452 addresses completed.
3718 out of 7452 addresses completed.
3719 out of 7452 addresses completed.
3720 out of 7452 addresses completed.
3721 out of 7452 addresses completed.
3722 out of 7452 addresses completed.
3723 out of 7452 addresses completed.
3724 out of 7452 addresses completed.
3725 out of 7452 addresses completed.
3726 out of 7452 addresses completed.
3727 out of 7452 addresses completed.
3728 out of 

3918 out of 7452 addresses completed.
3919 out of 7452 addresses completed.
3920 out of 7452 addresses completed.
3921 out of 7452 addresses completed.
3922 out of 7452 addresses completed.
3923 out of 7452 addresses completed.
3924 out of 7452 addresses completed.
3925 out of 7452 addresses completed.
3926 out of 7452 addresses completed.
3927 out of 7452 addresses completed.
3928 out of 7452 addresses completed.
3929 out of 7452 addresses completed.
3930 out of 7452 addresses completed.
3931 out of 7452 addresses completed.
3932 out of 7452 addresses completed.
3933 out of 7452 addresses completed.
3934 out of 7452 addresses completed.
3935 out of 7452 addresses completed.
3936 out of 7452 addresses completed.
3937 out of 7452 addresses completed.
3938 out of 7452 addresses completed.
3939 out of 7452 addresses completed.
3940 out of 7452 addresses completed.
3941 out of 7452 addresses completed.
3942 out of 7452 addresses completed.
3943 out of 7452 addresses completed.
3944 out of 

#### Add latitude and longitude to data frame

In [39]:
# add to room data
room_data["query_address"] = query_address
room_data["room_latitude"] = latitude
room_data["room_longitude"] = longitude

# see it
room_data

,room_title,woe_room_url,room_address,room_description,player_range,time_limit,difficulty_level,fear_level,age_requirement,success_rate,min_players,max_players,time_limit_str,minimum_age,difficulty_int,fear_int,query_address,room_latitude,room_longitude
0,"Escape room ""The Pirate's Curse"" by Escape Roo...",https://worldofescapes.com/washington/quests/e...,"3345 M Street NW, Washington, DC 20007","Ahoy, Matey! All aboard Ye Pirate Ship! All wh...",8-10,60,Very difficult,Not scary,5+,None,8,10,60 minutes,5,3,0,"3345 M St NW, Washington, DC 20007, USA",38.9053,-77.0675
1,"Escape room ""The Time Machine"" by MindTrix in ...",https://worldofescapes.com/portland/quests/mid...,"12000 SE 82nd Ave #1009 Happy Valley, OR 97086",Scientists are on the verge of discovering tim...,2-8,60,Difficult,Not scary,14+,None,2,8,60 minutes,14,2,0,"12000 SE 82nd Ave, Happy Valley, OR 97086, USA",45.4359,-122.576
2,"Escape room ""National Treasure"" by The Escape ...",https://worldofescapes.com/new-brunswick/quest...,"45 Lafayette Rd Fords, NJ 08863",While on vacation with a group of your buddies...,2-10,60,Very difficult,Not scary,NaN,None,2,10,60 minutes,nan,3,0,"45 Lafayette Rd, Fords, NJ 08863, USA",40.548,-74.3227
3,"Escape room ""The Lost Antidote"" by The Escape ...",https://worldofescapes.com/little-rock/quests/...,"1214 S Main St Little Rock, AR 72202","Dr. Stapleton, an evil scientist, has poisoned...",2-7,60,Average,Not scary,13+,0.4,2,7,60 minutes,13,1,0,"1214 S Main St, Little Rock, AR 72202, USA",34.7368,-92.2728
4,"Escape room ""Outbreak: Find the Cure"" by Maste...",https://worldofescapes.com/overland-park/quest...,"7810 West 151st Street Overland Park, KS 66223",The CDC is on lockdown after a deadly virus wa...,2-10,60,Difficult,Not scary,NaN,0.21,2,10,60 minutes,nan,2,0,"7810 151st St, Overland Park, KS 66223, USA",38.8551,-94.6762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7447,"Escape room ""Nuclear Shuttle"" by Escape Rooms ...",https://worldofescapes.com/dallas/quests/escap...,"1565 W Main ST #495 Lewisville, TX 75077",Your team is the second shift crew for the Orb...,2-8,60,Difficult,Not scary,13+,0.3,2,8,60 minutes,13,2,0,"1565 W Main St #495, Lewisville, TX 75067, USA",33.0436,-97.0342
7448,"Escape room ""The Wizard's Chamber"" by Escape N...",https://worldofescapes.com/warren-oh/quests/es...,"200 Chestnut Avenue Northeast Suite C Warren, ...",The founders created a vessel to tether their ...,4-8,60,Difficult,Not scary,10+,None,4,8,60 minutes,10,2,0,"200 Chestnut Ave NE Suite C, Warren, OH 44483,...",41.2377,-80.8088
7449,"Escape room ""Houdini's Magic Cell - Destinatio...",https://worldofescapes.com/andover/quests/all-...,"301 Mt Hope Ave Rockaway, NJ 07866","Just like Houdini began, you will start your q...",2-10,60,Difficult,Not scary,14+,None,2,10,60 minutes,14,2,0,"301 Mt Hope Ave, Rockaway, NJ 07866, USA",40.9071,-74.5534
7450,"Escape room ""High Noon in the Old West"" by Tim...",https://worldofescapes.com/atlanta/quests/time...,"1441 Dresden Dr Suite 230 Atlanta, GA 30319",It is 1899 and the final push to tame the wild...,1-10,60,Average,Not scary,12+,None,1,10,60 minutes,12,1,0,"1441 Dresden Dr #230, Atlanta, GA 30319, USA",33.8607,-84.3302


### Read review data .csv files and concatenate together

In [40]:
review_data = pd.concat([pd.read_csv(filepath_i) for filepath_i in glob.glob("data/reviews/*.csv")], ignore_index = True)

# See it
review_data

,room_title,woe_room_url,review_id,review
0,"Escape room ""Murder of Crows"" by Keynundrum in...",https://worldofescapes.com/topeka/quests/keynu...,0,Love the room design of Murder of Crows! Nice ...
1,"Escape room ""Murder of Crows"" by Keynundrum in...",https://worldofescapes.com/topeka/quests/keynu...,1,First time anyone in our group had done an esc...
2,"Escape room ""Murder of Crows"" by Keynundrum in...",https://worldofescapes.com/topeka/quests/keynu...,2,So much fun!!! Came so close to finishing the ...
3,"Escape room ""Immersive X"" by XscapeQuest in Sa...",https://worldofescapes.com/bay-city/quests/xsc...,0,None
4,"Escape room ""Captain's Maxwell's Shipwrecked"" ...",https://worldofescapes.com/milwaukee/quests/te...,0,"We were a group of 5 couples, with one couple ..."
...,...,...,...,...
19830,"Escape room ""Fatal Attraction"" by Cracked Norm...",https://worldofescapes.com/oklahoma-city/quest...,0,We did the Fatal Attraction room which was a c...
19831,"Escape room ""Fatal Attraction"" by Cracked Norm...",https://worldofescapes.com/oklahoma-city/quest...,1,I took five 12-13 year old girls to do Fatal A...
19832,"Escape room ""Fatal Attraction"" by Cracked Norm...",https://worldofescapes.com/oklahoma-city/quest...,2,We took my dad for Father's Day to Fatal Attra...
19833,"Escape room ""Chasm"" by Beat the Room in Sacram...",https://worldofescapes.com/sacramento/quests/b...,0,We had 6 family members working together and h...


### Read tag data .csv files and concatenate together

In [41]:
tag_data = pd.concat([pd.read_csv(filepath_i) for filepath_i in glob.glob("data/tags/*.csv")], ignore_index = True)

# See it
tag_data

,room_title,woe_room_url,tag_id,tag
0,"Escape room ""The Rec Room"" by Escape the Room ...",https://worldofescapes.com/detroit/quests/esca...,0,Public Ticketing
1,"Escape room ""The Rec Room"" by Escape the Room ...",https://worldofescapes.com/detroit/quests/esca...,1,Up to eight players
2,"Escape room ""The Play House"" by Escape Space G...",https://worldofescapes.com/portland/quests/esc...,0,Up to eight players
3,"Escape room ""The Chocolate Factory"" by Escape ...",https://worldofescapes.com/houston/quests/esca...,0,Public Ticketing
4,"Escape room ""The Chocolate Factory"" by Escape ...",https://worldofescapes.com/houston/quests/esca...,1,Up to ten players
...,...,...,...,...
22578,"Escape room ""The Butcher"" by Captured in Coven...",https://worldofescapes.com/trappe/quests/captu...,1,Detective
22579,"Escape room ""The Butcher"" by Captured in Coven...",https://worldofescapes.com/trappe/quests/captu...,2,Up to eight players
22580,"Escape room ""Swan Song"" by The SafeHouse in Tulsa",https://worldofescapes.com/tulsa/quests/the-sa...,0,Private Ticketing
22581,"Escape room ""Swan Song"" by The SafeHouse in Tulsa",https://worldofescapes.com/tulsa/quests/the-sa...,1,ADA compliant


## Export merged data frames to .csv files

In [43]:
# drop duplicatesa
room_data = room_data.drop_duplicates(inplace = False)
review_data = review_data.drop_duplicates(inplace = False)
tag_data = tag_data.drop_duplicates(inplace = False)

# replace None
# convert "None" to None
room_data = room_data.mask(room_data.eq("None"))
review_data = review_data.mask(review_data.eq("None"))
tag_data = tag_data.mask(tag_data.eq("None"))

# minimum age
room_data["minimum_age"] = [float(age) if age != "PG-13" else np.nan for age in room_data["minimum_age"]]

# write .csv
room_data.to_csv("data/room_data.csv", index = False)
review_data.to_csv("data/review_data.csv", index = False)
tag_data.to_csv("data/tag_data.csv", index = False)